## **주식 종목 코드를 입력 받아 각 재무지표를 계산하고 주식을 추천할 투자 성향을 출력**
- 일간 수익률
- 표준편차
- 평균 수익률
- 연간 수익률
- 샤프 지수
- 소르티노 지수
- 베타 계수
- 알파 계수

In [4]:
pip install finance-datareader

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr

def get_stock_code_by_name(stock_name, file_path):
    stock_df = pd.read_excel(file_path)
    code = stock_df.loc[stock_df['종목명'] == stock_name, '종목코드']
    if not code.empty:
        return code.values[0]
    else:
        raise ValueError("해당 종목명을 찾을 수 없습니다.")

def calculate_stock_metrics(stock_code, market_code, start_date, end_date):
    # 주가 데이터 수집
    stock = fdr.DataReader(stock_code, start=start_date, end=end_date)
    market = fdr.DataReader(market_code, start=start_date, end=end_date)

    # 일간 수익률 계산
    stock['Return'] = stock['Close'].pct_change().dropna()
    market['Return'] = market['Close'].pct_change().dropna()

    # 데이터프레임 병합하여 공통 날짜로 맞춤
    combined_data = pd.merge(stock[['Return']], market[['Return']], left_index=True, right_index=True, suffixes=('_stock', '_market')).dropna()

    # 표준편차
    std_dev = np.std(combined_data['Return_stock'])

    # 평균 수익률
    mean_return = np.mean(combined_data['Return_stock'])

    # 연간 수익률
    annual_return = (1 + mean_return) ** 252 - 1

    # 샤프비율
    annual_risk_free_rate = 3.51 / 100  # 현재 연간 무위험 수익률 3.51%
    daily_risk_free_rate = (1 + annual_risk_free_rate) ** (1/252) - 1  # 일일 무위험 수익률로 변환
    sharpe_ratio = (mean_return - daily_risk_free_rate) / std_dev

    # 소르티노 비율
    downside_std_dev = np.std(combined_data[combined_data['Return_stock'] < 0]['Return_stock'])
    sortino_ratio = (mean_return - daily_risk_free_rate) / downside_std_dev

    # 베타 계수
    covariance = np.cov(combined_data['Return_stock'], combined_data['Return_market'])[0, 1]
    market_variance = np.var(combined_data['Return_market'])
    beta = covariance / market_variance

    # 알파 계수
    market_mean_return = np.mean(combined_data['Return_market'])
    alpha = mean_return - (daily_risk_free_rate + beta * (market_mean_return - daily_risk_free_rate))
    
    return {
        'Standard Deviation': std_dev,
        'Mean Return': mean_return,
        'Annual Return': annual_return,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Beta': beta,
        'Alpha': alpha
    }

def classify_investment_style(metrics):
    sharpe_ratio = metrics['Sharpe Ratio']
    sortino_ratio = metrics['Sortino Ratio']
    std_dev = metrics['Standard Deviation']
    mean_return = metrics['Mean Return']
    beta = metrics['Beta']
    alpha = metrics['Alpha']
    annual_return = metrics['Annual Return']

    # Sharpe Ratio 기준
    if sharpe_ratio <= 0.5:
        sharpe_category = '안정'
    elif 0.5 < sharpe_ratio <= 1.0:
        sharpe_category = '균형'
    else:
        sharpe_category = '공격'

    # Sortino Ratio 기준
    if sortino_ratio < 0:
        sortino_category = '안정'
    elif 0 <= sortino_ratio <= 1:
        sortino_category = '균형'
    else:
        sortino_category = '공격'

    # Standard Deviation 기준
    overall_std_dev = np.std(metrics['Mean Return'])  # 전체 평균 표준편차
    if std_dev > overall_std_dev:
        std_dev_category = '공격'
    elif np.isclose(std_dev, overall_std_dev):
        std_dev_category = '균형'
    else:
        std_dev_category = '안정'

    # Mean Return 기준
    overall_mean_return = np.mean(metrics['Mean Return'])  # 전체 평균 수익률
    overall_std_dev_return = np.std(metrics['Mean Return'])  # 전체 평균 수익률의 표준편차
    if mean_return > overall_mean_return + overall_std_dev_return:
        mean_return_category = '공격'
    elif overall_mean_return - overall_std_dev_return <= mean_return <= overall_mean_return + overall_std_dev_return:
        mean_return_category = '균형'
    else:
        mean_return_category = '안정'

    # Annual Return 기준
    if annual_return > overall_mean_return + overall_std_dev_return:
        annual_return_category = '공격'
    elif overall_mean_return - overall_std_dev_return <= annual_return <= overall_mean_return + overall_std_dev_return:
        annual_return_category = '균형'
    else:
        annual_return_category = '안정'

    # Beta 기준
    if beta > 1:
        beta_category = '공격'
    elif 0.8 <= beta <= 1:
        beta_category = '균형'
    else:
        beta_category = '안정'

    # Alpha 기준
    if alpha <= 0:
        alpha_category = '안정'
    elif 0 < alpha <= 0.03:
        alpha_category = '균형'
    else:
        alpha_category = '공격'

    categories = [sharpe_category, sortino_category, std_dev_category, mean_return_category, beta_category, alpha_category, annual_return_category]
    category_counts = {'공격': 0, '균형': 0, '안정': 0}

    for category in categories:
        category_counts[category] += 1

    if category_counts['공격'] >= category_counts['균형'] and category_counts['공격'] >= category_counts['안정']:
        return '공격'
    elif category_counts['균형'] >= category_counts['안정']:
        return '균형'
    else:
        return '안정'

def analyze_stock(stock_name, market_code, start_date, end_date, file_path):
    stock_code = get_stock_code_by_name(stock_name, file_path)
    metrics = calculate_stock_metrics(stock_code, market_code, start_date, end_date)
    investment_style = classify_investment_style(metrics)
    return metrics, investment_style

# 종목명을 입력받기
stock_name = input("주식 종목명을 입력하세요: ")
market_code = '^KS11'
start_date = '2021-06-01'
end_date = '2024-05-31'
file_path = "./kospi_company.xlsx"

metrics, investment_style = analyze_stock(stock_name, market_code, start_date, end_date, file_path)
print(f"재무지표 : {metrics}")
print(f"투자성향 : {investment_style}")

재무지표 : {'Standard Deviation': 0.02221031377481121, 'Mean Return': 0.0005063768128522032, 'Annual Return': 0.13606968305989708, 'Sharpe Ratio': 0.01663508494449275, 'Sortino Ratio': 0.027788418539126957, 'Beta': 0.8026819946605287, 'Alpha': 0.0006556563594025246}
투자성향 : 균형
